# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

In [48]:
#useful tool for debugging
!pip install icecream

In [49]:
#all other imports 
import icecream as ic

In [ ]:
Utility Methods

In [9]:
azdias.OST_WEST_KZ.value_counts()



W    629528
O    168545
Name: OST_WEST_KZ, dtype: int64

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
#useful variables
addtional_customer_cols = ['PRODUCT_GROUP', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE']
azdias_dimensions = (891221,366)
azdias_num_rows = 891221
azdias_num_cols = 366
customers_num_rows = 191652
customers_num_cols = 369

In [3]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


###### observe warning above - Columns (18,19) have mixed types.  They need to be identified and addressed

In [4]:
#load the values and attributes
attributes_value_ranges = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header =1)
attributes_descriptions = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header = 1)

In [5]:
#check the correct rows and cols are loaded
print("azdias: ",azdias.shape)
print("customers: ",customers.shape)

assert azdias.shape[0] == azdias_num_rows, "azdias row count incorrect"
assert azdias.shape[1] == azdias_num_cols, "azdias col count incorrect"
assert customers.shape[0] == customers_num_rows, "azdias row count incorrect"
assert customers.shape[1] == customers_num_cols, "azdias col count incorrect"

#make a copy of the df
azdias_org = azdias
customers_org = customers

azdias:  (891221, 366)
customers:  (191652, 369)


In [6]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


In [5]:
azdias.to_csv('azdias.csv', index=False, sep=';')

In [5]:
customers.to_csv('customers.csv', index=False, sep=';')

In [77]:
pd.set_option('display.max_columns', None)
azdias.head(16)

LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0   910215        -1         NaN       NaN          NaN          NaN   
1   910220        -1         9.0       0.0          NaN          NaN   
2   910225        -1         9.0      17.0          NaN          NaN   
3   910226         2         1.0      13.0          NaN          NaN   
4   910241        -1         1.0      20.0          NaN          NaN   
5   910244         3         1.0      10.0          NaN          NaN   
6   910248        -1         9.0       0.0          NaN          NaN   
7   910261        -1         1.0      14.0          NaN          NaN   
8   645145        -1         9.0      16.0          NaN          NaN   
9   645153        -1         5.0      17.0          NaN          NaN   
10  645165         0         1.0      10.0          NaN          NaN   
11  645169        -1         NaN       NaN          NaN          NaN   
12  612558        -1         5.0      21.0          NaN          NaN   
13  612561        -1         8.0      20.0          NaN          NaN   
14  612565        -1         NaN       NaN          NaN          NaN   
15  612569         1         9.0      11.0          NaN          NaN   

    ALTER_KIND3  ALTER_KIND4  ALTERSKATEGORIE_FEIN  ANZ_HAUSHALTE_AKTIV  \
0           NaN          NaN                   NaN                  NaN   
1           NaN          NaN                  21.0                 11.0   
2           NaN          NaN                  17.0                 10.0   
3           NaN          NaN                  13.0                  1.0   
4           NaN          NaN                  14.0                  3.0   
5           NaN          NaN                  10.0                  5.0   
6           NaN          NaN                   NaN                  4.0   
7           NaN          NaN                  14.0                  6.0   
8           NaN          NaN                  16.0                  2.0   
9           NaN          NaN                  17.0                  9.0   
10          NaN          NaN                  10.0                  6.0   
11          NaN          NaN                   NaN                  NaN   
12          NaN          NaN                  14.0                  1.0   
13          NaN          NaN                  20.0                  2.0   
14          NaN          NaN                   NaN                  NaN   
15          NaN          NaN                  11.0                  1.0   

    ANZ_HH_TITEL  ANZ_KINDER  ANZ_PERSONEN  ANZ_STATISTISCHE_HAUSHALTE  \
0            NaN         NaN           NaN                         NaN   
1            0.0         0.0           2.0                        12.0   
2            0.0         0.0           1.0                         7.0   
3            0.0         0.0           0.0                         2.0   
4            0.0         0.0           4.0                         3.0   
5            0.0         0.0           1.0                         2.0   
6            0.0         0.0           1.0                         3.0   
7            0.0         0.0           1.0                         5.0   
8            1.0         0.0           1.0                         2.0   
9            0.0         0.0           1.0                         7.0   
10           0.0         0.0           1.0                         6.0   
11           NaN         NaN           NaN                         NaN   
12           0.0         0.0           2.0                         1.0   
13           0.0         0.0           1.0                         2.0   
14           NaN         NaN           NaN                         NaN   
15           0.0         0.0           1.0                         1.0   

    ANZ_TITEL  ARBEIT  BALLRAUM CAMEO_DEU_2015 CAMEO_DEUG_2015  \
0         NaN     NaN       NaN            NaN             NaN   
1         0.0     3.0       6.0             8A               8   
2         0.0     3.0       2.0             4C   

In [ ]:
customers.head(5)

In [ ]:
Initial Observations:
    
Warning on read_csv: Columns (18,19) have mixed types

LNR: first column: is it an index or identity? 

D19_LETZTER_KAUF_BRANCHE looks like it has other col names listed: check this maybe encode?
    
EINGEFUEGT_AM looks like a datetime object; no date times are listed in the info()

MIN_GEBAEUDEJAHR: looks like a year (jahr means year!): should be int not float
        
OST_WEST_KZ: has type object: can be encoded?

In [12]:
#look at non-numeric attributes #1
non_num_attributes_customers = customers.select_dtypes(include='object')
non_num_attributes_azdias = azdias.select_dtypes(include='object')


In [14]:
non_num_attributes_azdias.describe()

CAMEO_DEU_2015 CAMEO_DEUG_2015 CAMEO_INTL_2015  \
count          792242          792242          792242   
unique             45              19              43   
top                6B               8              51   
freq            56672           78023           77576   

       D19_LETZTER_KAUF_BRANCHE        EINGEFUEGT_AM OST_WEST_KZ  
count                    634108               798073      798073  
unique                       35                 5162           2  
top               D19_UNBEKANNT  1992-02-10 00:00:00           W  
freq                     195338               383738      629528

In [ ]:
#look at non-numeric attributes #2
non_num_attributes_azdias.head
#non_num_attributes.describe()

In [15]:
non_num_attributes_azdias.isna().sum().sort_values(ascending=False).head()


CAMEO_INTL_2015    50428
CAMEO_DEUG_2015    50428
CAMEO_DEU_2015     50428
OST_WEST_KZ        49927
EINGEFUEGT_AM      49927
dtype: int64

In [16]:
non_num_attributes_customers.isna().sum().sort_values(ascending=False).head()

CAMEO_INTL_2015    50428
CAMEO_DEUG_2015    50428
CAMEO_DEU_2015     50428
OST_WEST_KZ        49927
EINGEFUEGT_AM      49927
dtype: int64

In [ ]:
NB the same columns are listed

In [17]:
len(pd.unique(azdias['CAMEO_DEU_2015'])) #expect 44 plus 1 for NaN i.e. 45


46

In [13]:
pd.unique(azdias['CAMEO_DEU_2015'])

array([nan, '8A', '4C', '2A', '6B', '8C', '4A', '2D', '1A', '1E', '9D',
       '5C', '8B', '7A', '5D', '9E', '9B', '1B', '3D', '4E', '4B', '3C',
       '5A', '7B', '9A', '6D', '6E', '2C', '7C', '9C', '7D', '5E', '1D',
       '8D', '6C', '6A', '5B', '4D', '3A', '2B', '7E', '3B', '6F', '5F',
       '1C', 'XX'], dtype=object)

In [16]:
len(pd.unique(customers['CAMEO_DEU_2015'])) #expect 44 plus 1 for NaN i.e. 45


46

In [15]:
pd.unique(customers['CAMEO_DEU_2015'])

array(['1A', nan, '5D', '4C', '7B', '3B', '1D', '9E', '2D', '4A', '6B',
       '9D', '8B', '5C', '9C', '4E', '6C', '8C', '8A', '5B', '9B', '3D',
       '2A', '3C', '5F', '7A', '1E', '2C', '7C', '5A', '2B', '6D', '7E',
       '5E', '6E', '3A', '9A', '4B', '1C', '1B', '6A', '8D', '7D', '6F',
       '4D', 'XX'], dtype=object)

In [ ]:
#TODO: replace XX with NaN
#labelencode

In [33]:
pd.unique(customers['OST_WEST_KZ'])


array(['W', nan, 'O'], dtype=object)

In [23]:
#there are 891221 rows on the azdias df
len(pd.unique(azdias['LNR']))

891221

In [24]:
#there are 191652 rows on the customers df
len(pd.unique(customers['LNR']))

191652

In [ ]:
LNR is and id column and should be dropped

In [ ]:
EINGEFUEGT_AM_vals = azdias['EINGEFUEGT_AM'].unique()
print(len(EINGEFUEGT_AM_vals))
#print(sorted(EINGEFUEGT_AM_vals))
#print(sorted(EINGEFUEGT_AM_vals, reverse=True))
#error message: TypeError: '<' not supported between instances of 'float' and 'str'
print(EINGEFUEGT_AM_vals)
azdias.EINGEFUEGT_AM.head(600)

In [26]:
azdias['EINGEFUEGT_AM']= pd.to_datetime(azdias['EINGEFUEGT_AM'])
 
# Check the format of 'Date' column
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(267), int64(93), object(5)
memory usage: 2.4+ GB


In [28]:
customers['EINGEFUEGT_AM'].unique()

array(['1992-02-12 00:00:00', nan, '1992-02-10 00:00:00', ...,
       '2008-04-25 00:00:00', '2005-03-30 00:00:00', '2008-07-14 00:00:00'], dtype=object)

In [29]:
customers['EINGEFUEGT_AM']= pd.to_datetime(customers['EINGEFUEGT_AM'])
 
# Check the format of 'Date' column
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(267), int64(94), object(7)
memory usage: 539.5+ MB


In [ ]:
#EINGEZOGENAM_HH_JAHR is a year feature.  it should be int same as e.g GEBURTSJAHR


In [ ]:
#search for gros and fein

In [ ]:
dias_values["Attribute"] = dias_values["Attribute"].ffill()
dias_values.head(10)

In [10]:
def printListDetails(l, desc='no description', counter=1, to_sort=False):
    '''
    '''
    
    if to_sort:
        new_list = [str(i) for i in l]
        l = sorted(new_list)
        
    print (counter, desc, ":", 'len:', len(l), ',', l, '\n')
    counter = counter + 1
    return counter

In [ ]:
#Analyse columns
azdias_cols = azdias.columns
customers_cols = customers.columns
attributes_ranges_cols_list = attributes_value_ranges.Attribute
attributes_descriptions_cols_list = attributes_descriptions.Attribute

diff_customers_cols_versus_azdias_cols = list(set(customers_cols) - set(azdias_cols))
diff_azdias_cols_versus_customers_cols = list(set(azdias_cols) - set(customers_cols))
diff_attrib_ranges_cols_versus_customers_cols = list(set(attributes_ranges_cols_list) - set(customers_cols))
diff_customers_cols_versus_attrib_ranges_cols = list(set(customers_cols) - set(attributes_ranges_cols_list))
diff_attrib_ranges_cols_versus_azdias_cols = list(set(attributes_ranges_cols_list) - set(azdias_cols))
diff_azdias_cols_versus_attrib_ranges_cols = list(set(azdias_cols) - set(attributes_ranges_cols_list))
diff_between_missing_col_customers_azdias = list(set(diff_attrib_ranges_cols_versus_customers_cols) - set(diff_attrib_ranges_cols_versus_azdias_cols))
diff_between_missing_col_azdias_customers = list(set(diff_attrib_ranges_cols_versus_azdias_cols) - set(diff_attrib_ranges_cols_versus_customers_cols))

printListDetails(diff_customers_cols_versus_azdias_cols, "Customer versus azdias lists (expect 3 cols)", counter,to_sort=True)
printListDetails(diff_azdias_cols_versus_customers_cols, "Azdias versus customers lists (expect 0 cols)", counter,to_sort=True)
printListDetails(diff_attrib_ranges_cols_versus_customers_cols, "Attributes versus customers lists", counter,to_sort=True)
printListDetails(diff_customers_cols_versus_attrib_ranges_cols, "Customers versus Attributes lists", counter,to_sort=True)
printListDetails(diff_attrib_ranges_cols_versus_azdias_cols, "Attributes versus azdias lists", counter,to_sort=True)
printListDetails(diff_azdias_cols_versus_attrib_ranges_cols, "Azdias versus Attributes lists", counter,to_sort=True)
printListDetails(diff_between_missing_col_customers_azdias, "Excess missing cols customers versus azdias (expect 0 cols)", counter,to_sort=True)
printListDetails(diff_between_missing_col_azdias_customers, "Excess missing cols azdias versus customers (expect 0 cols):", counter,to_sort=True)


In [25]:
print(diff_customers_cols_versus_azdias_cols)
diff_customers_cols_versus_azdias_cols=sorted(diff_customers_cols_versus_azdias_cols)
print(diff_customers_cols_versus_azdias_cols)

['ONLINE_PURCHASE', 'CUSTOMER_GROUP', 'PRODUCT_GROUP']
['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']


In [ ]:
1. This is expected: we were told that the customer df has three extra columns
    
2. Expected: the azdias df does not have more cols than the customers df

3. Surprising: there are 43 more attribs listed than are used on the customers df

4. Surprising: there are 97 cols on the customers df that are not described in the attribs listed

5. Surprising: there are 43 more attribs listed than are used on the azdias df

6. Surprising: there are 94 cols on the azdias df that are not described in the attribs listed

7. Expected: the 'missing' cols are consistent 

8. Expected: the 'missing' cols are consistent 

In [109]:
def find_terms_in_list(llist, term):
    '''
    Searches a list for a given term
    '''
    list_of_terms = []
    for i in llist:
        #print(i)
        if (term.upper() in str(i).upper()):
            list_of_terms.append(i)

    return list_of_terms

#grob_and_fein_list = find_terms_in_list( attributes_value_ranges.Attribute, 'fein') + find_terms_in_list( attributes_value_ranges.Attribute, 'grob')
#print(sorted(grob_and_fein_list))


In [112]:
print(find_terms_in_list(attributes_value_ranges.Meaning.unique(), 'known'))

['unknown', 'unknown / no main age detectable', 'no transactions known', 'no transaction known', 'residental building buildings without actually known household', 'mixed building without actually known household or comapny ', 'company building w/o known company ', 'mixed building without actually known household ', 'mixed building without actually known company ']


In [59]:
unknown_terms = [ 'unknown / no main age detectable', 'no transactions known', 'no transaction known', 'unknown']

In [ ]:
All of the above terms appear on the first row of each new attribute on the 'DIAS Attributes - Values 2017.xlsx'
There is a corresponding value for the unknown value e.g. 0,-1,-9 etc. Sometimes two values represent unknown e.g. 0,1
The first task is to get a list of the attributes that use these terms
The second task is to traverse the df and replace the values with np.nan - this brings consistency to the representation 
of unknown and will make it easier to judge if columns/rows should be kept or dropped

In [71]:
counter=0
for term in unknown_terms:
    if counter == 0:      
        df = attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == term]
        counter = 1
    else:
        df = pd.concat([df,attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == term]])
#print(df.shape)
attribute_list = df['Attribute'].tolist()
#print(attribute_list)

In [73]:
def count_nan_vals_in_col(df, col):
    '''
    Takes a df and a col name and
    returns the number of nan valus
    '''
    return df[col].isnull().sum()

In [74]:
def check_number_of_rows_with_nan_in_specific_cols(df,col1, col2):
    '''
    Method takes a df and two col names, checks if np.nan
    is in both, specified cols of the same row and then returns a count
    '''
    list_of_coincidences = list(np.where((df[col1]==df[col2]) | ((df[col1]!=df[col1]) & (df[col2]!=df[col2])), df[col2],'other'))
    return list_of_coincidences.count(np.nan)

d = {'Type': [np.nan,np.nan,'C',np.nan], 'Set': [np.nan,'Z','X',np.nan]}
df = pd.DataFrame(data=d)

assert check_number_of_rows_with_nan_in_specific_cols(df, 'Set','Type') == 2

In [96]:
#df = azdias.loc[azdias['LP_FAMILIE_FEIN'].isnull() & azdias['LP_FAMILIE_GROB'].isnull()]
#df = azdias.loc[azdias['LP_STATUS_FEIN'].isnull() & azdias['LP_STATUS_GROB'].isnull()]
df = azdias.loc[azdias['LP_LEBENSPHASE_FEIN'].isnull() & azdias['LP_LEBENSPHASE_GROB'].isnull()]

df = customers.loc[customers['LP_FAMILIE_FEIN'].isnull() & customers['LP_FAMILIE_GROB'].isnull()]
#df = customers.loc[customers['LP_STATUS_FEIN'].isnull() & customers['LP_STATUS_GROB'].isnull()]
#df = customers.loc[azdias['LP_LEBENSPHASE_FEIN'].isnull() & customers['LP_LEBENSPHASE_GROB'].isnull()]
df.shape

(3213, 369)

In [75]:
print('###### LP_FAMILIE_FEIN/GROB ######')
print()

print('Number of nans in LP_FAMILIE_GROB in azdias df: ', count_nan_vals_in_col(azdias, 'LP_FAMILIE_GROB'))
print('Number of nans in LP_FAMILIE_FEIN in azdias df: ', count_nan_vals_in_col(azdias, 'LP_FAMILIE_FEIN'))
print('Number of rows where LP_FAMILIE_FEIN/GROB both nan in azdias df: ', \
      check_number_of_rows_with_nan_in_specific_cols(azdias, 'LP_FAMILIE_GROB','LP_FAMILIE_FEIN')
     )
print('Unique values in LP_FAMILIE_GROB in azdias df: ', sorted(azdias['LP_FAMILIE_GROB'].unique()))
print('Unique values in LP_FAMILIE_FEIN in azdias df: ', sorted(azdias['LP_FAMILIE_FEIN'].unique()))
print('Unique value counts in LP_FAMILIE_GROB in azdias df: \n', azdias.LP_FAMILIE_GROB.value_counts())
print('Unique value counts in LP_FAMILIE_FEIN in azdias df: \n', azdias.LP_FAMILIE_FEIN.value_counts())
print()

print('Number of nans in LP_FAMILIE_GROB in customers df: ', count_nan_vals_in_col(customers, 'LP_FAMILIE_GROB'))
print('Number of nans in LP_FAMILIE_FEIN in customers df: ', count_nan_vals_in_col(customers, 'LP_FAMILIE_FEIN'))
print('Number of rows where LP_FAMILIE_FEIN/GROB both nan in customers df: ', \
      check_number_of_rows_with_nan_in_specific_cols(customers, 'LP_FAMILIE_GROB','LP_FAMILIE_FEIN')
     )
print()

print('###### LP_STATUS_FEIN/GROB ######')
print()

print('Number of nans in LP_STATUS_GROB in azdias df: ', count_nan_vals_in_col(azdias, 'LP_STATUS_GROB'))
print('Number of nans in LP_STATUS_FEIN in azdias df: ', count_nan_vals_in_col(azdias, 'LP_STATUS_FEIN'))
print('Number of rows where LP_STATUS_FEIN/GROB both nan in azdias df: ', \
      check_number_of_rows_with_nan_in_specific_cols(azdias, 'LP_STATUS_GROB','LP_STATUS_FEIN')
     )
print('Unique values in LP_STATUS_GROB in azdias df: ', sorted(azdias['LP_STATUS_GROB'].unique()))
print('Unique values in LP_STATUS_FEIN in azdias df: ', sorted(azdias['LP_STATUS_FEIN'].unique()))
print()

print('Number of nans in LP_STATUS_GROB in customers df: ', count_nan_vals_in_col(customers, 'LP_STATUS_GROB'))
print('Number of nans in LP_STATUS_FEIN in customers df: ', count_nan_vals_in_col(customers, 'LP_STATUS_FEIN'))
print('Number of rows where LP_STATUS_FEIN/GROB both nan in customers df: ', \
      check_number_of_rows_with_nan_in_specific_cols(customers, 'LP_STATUS_GROB','LP_STATUS_FEIN')
     )
print()

print('###### LP_LEBENSPHASE_FEIN/GROB ######')
print()

print('Number of nans in LP_LEBENSPHASE_GROB in azdias df: ', count_nan_vals_in_col(azdias, 'LP_LEBENSPHASE_GROB'))
print('Number of nans in LP_LEBENSPHASE_FEIN in azdias df: ', count_nan_vals_in_col(azdias, 'LP_LEBENSPHASE_FEIN'))
print('Number of rows where LP_LEBENSPHASE_FEIN/GROB both nan in azdias df: ', \
      check_number_of_rows_with_nan_in_specific_cols(azdias, 'LP_LEBENSPHASE_GROB','LP_LEBENSPHASE_FEIN')
     )
print('Unique values in LP_LEBENSPHASE_GROB in azdias df: ', sorted(azdias['LP_LEBENSPHASE_GROB'].unique()))
print('Unique values in LP_LEBENSPHASE_FEIN in azdias df: ', sorted(azdias['LP_LEBENSPHASE_FEIN'].unique()))
print('Unique value counts in LP_LEBENSPHASE_GROB in azdias df: \n', azdias.LP_LEBENSPHASE_GROB.value_counts())
print('Unique value counts in LP_LEBENSPHASE_FEIN in azdias df: \n', azdias.LP_LEBENSPHASE_FEIN.value_counts())
print()

print()

print('Number of nans in LP_LEBENSPHASE_GROB in customers df: ', count_nan_vals_in_col(customers, 'LP_LEBENSPHASE_GROB'))
print('Number of nans in LP_LEBENSPHASE_FEIN in customers df: ', count_nan_vals_in_col(customers, 'LP_LEBENSPHASE_FEIN'))
print('Number of rows where LP_LEBENSPHASE_FEIN/GROB both nan in customers df: ', \
      check_number_of_rows_with_nan_in_specific_cols(customers, 'LP_LEBENSPHASE_GROB','LP_LEBENSPHASE_FEIN')
     )
print()

###### LP_FAMILIE_FEIN/GROB ######

Number of nans in LP_FAMILIE_GROB in azdias df:  4854
Number of nans in LP_FAMILIE_FEIN in azdias df:  4854
Number of rows where LP_FAMILIE_FEIN/GROB both nan in azdias df:  0
Unique values in LP_FAMILIE_GROB in azdias df:  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, nan]
Unique values in LP_FAMILIE_FEIN in azdias df:  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, nan]
Unique value counts in LP_FAMILIE_GROB in azdias df: 
 1.0    426379
5.0    200780
2.0    104305
0.0     72938
4.0     52784
3.0     29181
Name: LP_FAMILIE_GROB, dtype: int64
Unique value counts in LP_FAMILIE_FEIN in azdias df: 
 1.0     426379
10.0    137913
2.0     104305
0.0      72938
11.0     51719
8.0      23032
7.0      20730
4.0      12303
5.0      11920
9.0      11148
6.0       9022
3.0       4958
Name: LP_FAMILIE_FEIN, dtype: int64

Number of nans in LP_FAMILIE_GROB in customers df:  3213
Number of nans in LP_FAMILIE_FEIN in customers df:  3213
Number of rows where LP_FAM

In [ ]:
The _FEIN column is a more granular representation of the _GROB column.  There is no need for both columns.  
No row, for any of the three features, has both _FEIN & _GROB column set to NaN.  
Therefore, right a function to ensure _GROB column is fully populated and drop the _FEIN

In [ ]:
def recode_LP_FAMILIE_GROB(df):
    '''
    '''
    mapping_dict = {1:1,2:1,
                    3:4,4:4,5:4,
                    6:7,7:7,8:7,
                    9:10,10:10,11:10
                   }
    df['LP_FAMILIE_GROB'] = df['LP_FAMILIE_FEIN'].apply(lambda x: mapping_dict.get( x, np.nan ) if (np.isnan(x)) else x) #dict.get( arg, resp_if_no_match )
    return df

df = azdias.tail(20)
df1=df[['LP_FAMILIE_GROB','LP_FAMILIE_FEIN']]
print(df1)
#df=recode_LP_FAMILIE_GROB(df)
#df1=df[['LP_FAMILIE_GROB','LP_FAMILIE_FEIN']]
#print(df1)

In [ ]:
def encode_OST_WEST_KZ(df):
    '''
    Update the xxx col
    Map O->1, W->2
    Return updated df
    '''
    mapping_dict = {'O':1,'W':2}
    df['OST_WEST_KZ'] = df['OST_WEST_KZ'].apply(lambda x:mapping_dict.get( x, np.nan )) #dict.get( arg, resp_if_no_match )
    return df

df=azdias.tail(100)    
print(df['OST_WEST_KZ'])
encode_OST_WEST_KZ(df)
print(df['OST_WEST_KZ'])

In [60]:
#this can be deleted
unknown_terms = []
for i in attributes_value_ranges.Meaning.unique():
    if ('KNOWN' in str(i).upper()):
        unknown_terms.append(i)

print(unknown_terms)  



['unknown', 'unknown / no main age detectable', 'no transactions known', 'no transaction known', 'residental building buildings without actually known household', 'mixed building without actually known household or comapny ', 'company building w/o known company ', 'mixed building without actually known household ', 'mixed building without actually known company ']


In [63]:
print(find_terms_in_list(attributes_value_ranges.Meaning.unique(), 'known'))

['unknown', 'unknown / no main age detectable', 'no transactions known', 'no transaction known', 'residental building buildings without actually known household', 'mixed building without actually known household or comapny ', 'company building w/o known company ', 'mixed building without actually known household ', 'mixed building without actually known company ']


In [24]:
def value_range_for_attribute(df, sKey_Name):
    '''
    '''
    value_list = []
    cell_value = str(df.loc[df["Attribute"] == sKey_Name, "Value"].values[0]).split(',')
    for i in cell_value:
        value_list.append(int(i))
    
    return value_list

[-1]


In [ ]:
#azdias_sub.BIP_FLAG.unique()
list(azdias_sub.columns)

In [ ]:
def replace_unknown_values_with_nan(df, list_of_attribs):
    '''
    '''
    attribs_not_present = []
    for attribute in list_of_attribs:
        print("replace_unknown_values_with_nan:", attribute)
        #df['CustomRating'] = df.apply(lambda x: custom_rating(x['Genre'],x['Rating']),axis=1)
        #nan_counts['percent_nan_missing'] = nan_counts[0].apply(lambda x: x*100/len(df))
        #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
        #df[attribute] = df[attribute].apply(lambda x:update_cell_to_nan_for_unknown(attribute, x))
        #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
        #df[attrib] = df[attrib].apply(replace_unkown_with_nan, args=(unkown))
        #df[attribute] = df[attribute].apply(update_cell_to_nan_for_unknown, args=(attribute))
        if attribute in df.columns:
            df[attribute] = df.apply(lambda x:update_cell_to_nan_for_unknown(attribute, x[attribute]),axis=1)
        else:
            attribs_not_present.append(attribute)

In [ ]:
pd.set_option('display.max_columns', None)
#azdias_sub = azdias.head(15)
print(attributes_with_unknowns)
attribute_list = attributes_with_unknowns['Attribute'].tolist()
print(attribute_list)
#azdias_sub



def update_cell_to_nan_for_unknown(colname, curr_value):
    print("update_cell_to_nan_for_unknown: ", colname)
    print("update_cell_to_nan_for_unknown: ", curr_value)
    #if curr_value.isin(value_range_for_attribute(attributes_with_unknowns,colname)):
    if curr_value in (value_range_for_attribute(attributes_with_unknowns,colname)):
        return np.nan
    else:
        return curr_value

#this is working
#print(update_cell_to_nan_for_unknown("AGER_TYP",-1))

#azdias["AGER_TYP"] = azdias["AGER_TYP"].apply(replace_unkown_with_nan, args=(ager_typ_unkwon))
#azdias_sub["AGER_TYP"] = azdias["AGER_TYP"].apply(update_cell_to_nan_for_unknown, args=(value_range_for_attribute("AGER_TYPE")))
#replace_unknown_values_with_nan(azdias_sub, attribute_list)
#azdias_sub

In [ ]:
#jotter

#df = pd.DataFrame([[1, 2, np.nan, 4], [np.nan, np.nan, 3, 4],[1, 2, 3], [np.nan, np.nan, np.nan, 4]])
df = pd.DataFrame([[1, 2, np.nan, 4], [np.nan, np.nan, np.nan, 4],[1, 2, 3], [np.nan, 2, np.nan, 4]])
#%timeit -n 1000 df.isnull().sum(axis=1)
#x = pd.DataFrame(df.isnull().sum(axis=1))
#y = list(df.isnull().sum(axis=1))
#x = df.isnull().sum(axis=1)
#print(y)
#print(x)
#print(list(x.iloc[:,-1]))
#print(calculate_percentage_nan_per_row(df))
print(df)
print(remove_rows_with_nan_above_threshold(df,50))


In [49]:
d = {'col1': [1.0, 2], 'col2': [3., 4]}
df = pd.DataFrame(data=d)
#df.dtypes
#df
# col1    int64
# col2    int64
# dtype: object
# Cast all columns to int32:

#df.col1.astype('int').dtypes
#df.col1=df.col1.astype('int32').dtypes
#df=df.astype('int32').dtypes

df = df.astype({"col1": int})
df
# col1    int32
# col2    int32
# dtype: object

col1  col2
0     1   3.0
1     2   4.0

In [27]:
def remove_rows_with_nan_above_threshold(df, threshold):
    '''
    '''   
    df=calculate_percentage_nan_per_row(df, True)
    df = df.drop(df[df.percent_nan > threshold].index) 
    df = df.drop('percent_nan',  axis='columns')
    
    return df

In [ ]:
def calculate_percentage_nan_per_row(df, add_col_to_df=False):
    '''
    '''
    rowcount = df.shape[1] 
    nan_count = list(df.isnull().sum(axis=1)) #just check is this counting nans or nulls???
    if add_col_to_df:
        df = df.assign(percent_nan = nan_count) 
        df['percent_nan'] = df['percent_nan'].apply(lambda x: x*100/rowcount)
        return df
    else:
        return pd.Dataframe(nan_count)
    
pd.set_option('display.max_columns', 5)
azdias_sub = azdias.head(15)
print(azdias_sub)
print()
azdias_sub = remove_rows_with_nan_above_threshold(azdias_sub, threshold = 1.9)
#zdias_sub = calculate_percentage_nan_per_row(azdias_sub, True)
print(azdias_sub)

In [13]:
#find the cols with mixed data types
def find_columns_with_mixed_data_dtypes(df):
    '''
    '''
    columnsWithMixedDataTypes = []
    for col in df.columns:
        weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis=1)
        if len(df[weird]) > 0:
            #print(col)
            columnsWithMixedDataTypes.append(col)
    
    return columnsWithMixedDataTypes

In [6]:
#Error message on loading: DtypeWarning: Columns (18,19) have mixed types
#=> these are the only two we need to worry about
#first check do they appear in the lists above
print(azdias.columns[18:20])
print(customers.columns[18:20])

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')
Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')


In [13]:
mixed_dtype_colnames = ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015']
#note: CAMEO_INTL_2015 is CAMEO_DEUINTL_2015 on the Attributes s/s

In [ ]:
#look at values in these warning cols
print(pd.value_counts(azdias.CAMEO_DEUG_2015))
print(pd.value_counts(azdias.CAMEO_INTL_2015))

print(pd.value_counts(customers.CAMEO_DEUG_2015))
print(pd.value_counts(customers.CAMEO_INTL_2015))

In [23]:
#method to update these two columns
#NB CAMEO_INTL_2015 is CAMEO_DEINTL_2015 in DIAS Attribute Values 2017 s/s 
#CAMEO_INTL_2015: object type can be set to -1
#CAMEO_DEUG_2015: object type can be set to -1
#also will cast ALL values to ints

def update_mixed_data_type_columns(df, list_of_cols):
    '''
    '''  
    for col in list_of_cols:
        #first replace the object types with -1
        #df[col].replace({"X": -1, "XX": -1}, inplace=True)
        #first replace the object types with np.nan
        df[col].replace({"X": np.nan, "XX": np.nan}, inplace=True)
        #set Nan to -1
        df[col] = df[col].fillna(-1)
        #cast everything to int
        df[col] = df[col].astype(int)   

In [14]:
updateMixedDataTypeColumns(azdias, mixed_dtype_colnames)
updateMixedDataTypeColumns(customers, mixed_dtype_colnames)

In [15]:
def calc_percent_nan_missing(df):
    '''
    '''
    
    #get a count of the nans
    nan_counts = pd.DataFrame(df.isna().sum())
    #calculate percent_nan_missing
    nan_counts['percent_nan_missing'] = nan_counts[0].apply(lambda x: x*100/len(df))
    #reset the indexes
    nan_counts.reset_index(level=0, inplace=True)
    #set column names
    nan_counts.columns = ['feature', 'count', 'percent_nan_missing']
    #drop sum column
    nan_counts=nan_counts.drop(columns=['count'])
    
    return nan_counts

In [16]:
azdias_nan_cols = calc_percent_nan_missing(azdias)
customers_nan_cols = calc_percent_nan_missing(customers)

In [18]:
def graph_compare_nan_percentages(df1, df1_name, df2, df2_name):
    """
    """
    fig = plt.figure(figsize=(10,80))

    fig.suptitle("NaN %s")

    ax = fig.add_subplot(121)
    ax.set_title(df1_name)
    sns.barplot(y="feature", x="percent_nan_missing", data=df1[df1.percent_nan_missing>=0], ax= ax)

    ax = fig.add_subplot(122)
    ax.set_title(df2_name)
    sns.barplot(y="feature", x="percent_nan_missing", data=df2[df2.percent_nan_missing>=0], ax= ax)

    #fig.tight_layout(rect=[0, 0.03, 1, 0.975])
    fig.tight_layout()
    plt.show()

In [ ]:
graph_compare_nan_percentages(azdias_nan_cols, 'azdias', customers_nan_cols, 'customers')

In [22]:
def graph_distribution_of_num_attributes(df):
    '''
    '''
    num_attributes = df.select_dtypes(exclude='object')

    fig = plt.figure(figsize=(12,18))
    for i in range(len(num_attributes.columns)):
        fig.add_subplot(9,4,i+1)
        sns.distplot(num_attributes.iloc[:,i].dropna())
        plt.xlabel(num_attributes.columns[i])

    plt.tight_layout()
    plt.show()

In [ ]:
graph_distribution_of_num_attributes(azdias)

In [ ]:
def column_list_of_nans_below_threshold(df,threshold):
    l = []
    l = list(df.drop(df.loc[:,list((100*(df.isnull().sum()/len(df.index))>=threshold))].columns, 1).columns.values)
    print("# Columns having more than %s percent missing values:"%threshold,(df.shape[1] - len(l)))
    print("Columns:\n",list(set(list((df.columns.values))) - set(l)))
    return l

In [ ]:
#look at the numeric attributes #1
num_attributes = customers.select_dtypes(exclude='object')
num_attributes.describe()

In [ ]:
#look at the numeric attributes #2
num_attributes.isna().sum().sort_values(ascending=False).head()

In [ ]:
#Comprehensive method to do all data cleaning
def clean_df(df):
    '''
    '''

    #address Columns (18,19) have mixed types [CAMEO_DEUG_2015, CAMEO_INTL_2015]

    #address WOHNLAGE col
    
    #address AGER_TYP col
    
    #resolve conflict fein v grob for 3 cols [LP_FAMILIE_FEIN/GROB, LP_LEBENSPHASE_FEIN/GROB, LP_STATUS_FEIN/GROB]
      
    #cast EINGEFUEGT_AM to datetime
    
    #cast to EINGEZOGENAM_HH_JAHR to int similar to GEBURTSJAHR
    
    #get list of columns to check for 'unknowns'
    
    #traverse df and update unknown to NaN
    
    #encode D19_LETZTER_KAUF_BRANCHE
    
    #encode OST_WEST_KZ
    
    #encode CAMEO_DEU_2015
    
    #drop cols explicitly chosen (LNR,....)
    
    #drop columns with high % of NaNs
    
    #drop rows with high % of NaNs
    

    return df

In [ ]:
#Call clean_df(df)
azdias_clean = clean_df(azdias)
customers_clean = clean_df(customers)

In [ ]:
#Scale the transformed data

In [ ]:
#Save the data ready for the next phase

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')